# Color Image Quantization
Reduce the number of colors in images (`lena.png`, `peppers.tif`) to 16 using K-means clustering, implemented from scratch. Log L2 norms per iteration and optionally visualize quantized images.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

def kmeans_quantization(image, k=16, max_iterations=100, visualize=False):
    orig_shape = image.shape
    pixels = image.reshape(-1, 3)
    num_pixels = pixels.shape[0]

    l2_norms = []

    rng = np.random.default_rng()
    centroid_indices = rng.choice(num_pixels, size=k, replace=False)
    cents = pixels[centroid_indices]

    if visualize:
        plt.figure(figsize=(8, 4))

    for i in range(max_iterations):
        distances_sq = np.sum((pixels[:, np.newaxis, :] - cents[np.newaxis, :, :])**2, axis=2)

        labels = np.argmin(distances_sq, axis=1)

        new_cen = np.zeros((k, 3))
        for j in range(k):
            as_pixels = pixels[labels == j]
            if len(as_pixels) > 0:
                new_cen[j] = np.mean(as_pixels, axis=0)
            else:
                new_cen[j] = pixels[rng.choice(num_pixels)]
        if visualize:
            plt.clf()

            plt.subplot(1, 2, 1)
            dis_image = cents[labels].reshape(orig_shape)
            plt.imshow(dis_image)
            plt.title(f'Quantized - Iteration {i+1}')
            plt.axis('off')
            plt.draw()
            plt.pause(0.1)

        if np.allclose(cents, new_cen):
            print(f"Converged at iteration {i+1}.")
            break

        cents = new_cen

        quantized_pixels_iter = cents[labels]
        quantized_image_iter = quantized_pixels_iter.reshape(orig_shape)

        current_l2 = calculate_l2_norm(image, quantized_image_iter)
        l2_norms.append(current_l2)

    final_quantized_pixels = cents[labels]
    quantized_image = final_quantized_pixels.reshape(orig_shape)

    return quantized_image, l2_norms

In [ ]:
def calculate_l2_norm(original, quantized):
    l2_norm = np.linalg.norm(original - quantized)
    return l2_norm

In [ ]:
lena_path = 'lena.png'
lena = cv2.imread(lena_path).astype(np.float32) / 255.0
lena = cv2.cvtColor(lena, cv2.COLOR_BGR2RGB)


k = 16
max_iterations = 100
visualize = False

In [ ]:
lena_orig = cv2.imread(lena_path).astype(np.float32) / 255.0
plt.imshow(cv2.cvtColor(lena_orig, cv2.COLOR_BGR2RGB))
plt.title('original Lena')
plt.axis('off')
plt.show()
lena_quantized, lena_l2_norms = kmeans_quantization(lena, k, max_iterations)

with open('L2_norm_log.txt', 'w') as f:
    f.write('Lena L2 Norms:\n' + '\n'.join(map(str, lena_l2_norms)) + '\n')

lena_l2 = calculate_l2_norm(lena, lena_quantized)

lena_quantized_uint8 = (lena_quantized * 255).astype(np.uint8)
flag = 1
if flag:
  plt.imshow(lena_quantized_uint8[:, :, ::1])
  plt.title('Lena Quantized')
  plt.axis('off')
  plt.show()

  print(f'Total L2 Norm: {lena_l2:.2f}')

  cv2.imwrite('lena_quantized.png', lena_quantized_uint8)

Output hidden; open in https://colab.research.google.com to view.